In [ ]:
import numpy as np
import ectopylasm as ep

import symfit as sf

import ipyvolume as ipv

In [ ]:
cone = ep.geometry.Cone(0.5, 0.5)
n_steps = 20
xyz = np.array(ep.geometry.cone_surface(cone, n_steps=n_steps))
xyz = xyz.reshape(3, n_steps*n_steps)
xyz += np.random.normal(0, 0.03, xyz.shape)

In [ ]:
h, radius, rot_x, rot_y, bx, by, bz = sf.parameters('h, radius, rot_x, rot_y, bx, by, bz')
x, y, z, f = sf.variables('x, y, z, f')

x_min_b = sf.Matrix([x - bx, y - by, z - bz])  # column matrix

M_x = sf.Matrix([[1, 0, 0],
                 [0, sf.cos(-rot_x), sf.sin(-rot_x)],
                 [0, -sf.sin(-rot_x), sf.cos(-rot_x)]])

M_y = sf.Matrix([[sf.cos(-rot_y), 0, sf.sin(-rot_y)],
                 [0, 1, 0],
                 [-sf.sin(-rot_y), 0, sf.cos(-rot_y)]])

u = M_x @ (M_y @ x_min_b)

cone_model = {
    f: h**2 * (u[0]**2 + u[1]**2) / radius**2 - (u[2] - h)**2
}

In [ ]:
cone_fit = sf.Fit(cone_model, x=xyz[0], y=xyz[1], z=xyz[2], f=np.zeros_like(xyz[0]))

cone_fit_result = cone_fit.execute()

print(cone_fit_result)

In [ ]:
cone_constraints = [
    sf.LessThan(sf.Abs(u[2]), h)
]

cone_fit = sf.Fit(cone_model, x=xyz[0], y=xyz[1], z=xyz[2], f=np.zeros_like(xyz[0]),
                  constraints=cone_constraints)

cone_fit_result = cone_fit.execute()

In [ ]:
u

In [ ]:
u[2]

In [ ]:
print(cone_fit_result)

In [ ]:
def plot_cone_fit(result):
    cone = ep.geometry.Cone(result.params['h'], result.params['radius'],
                            rot_x=result.params['rot_x'], rot_y=result.params['rot_y'],
                            base_pos=ep.geometry.Point(
                                result.params['bx'],
                                result.params['by'],
                                result.params['bz']
                            ))
    fig = ep.visualize.plot_cone(cone)
    return fig

In [ ]:
ipv.clear()
ipv.scatter(*xyz)
plot_cone_fit(cone_fit_result)
ipv.show()